<h1>Libraries And Utilities</h1>

In [ ]:
!pip3 install lime

In [ ]:
import os
import pathlib
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lime import lime_image
from skimage.segmentation import mark_boundaries
from skimage.io import imread, imsave
from skimage.util import crop
from skimage.transform import resize

from sklearn.metrics import confusion_matrix,precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_input_inceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.preprocessing import image_dataset_from_directory, image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.data.experimental import cardinality

from google.colab import drive
drive.mount('/content/drive')

<h1>Load Dataset</h1>

In [ ]:
!unzip -q '/content/drive/MyDrive/tese_mestrado_JoseMauricio_ISEC/datasets/dataset_CD_UC_v5.zip'

<h1>Pre-processing</h1>

In [ ]:
#Crop images
for url in glob.glob('/content/dataset_CD_UC/ulcerative-colitis-crop/*.jpg'):
  img = imread(url)
  crop_img = crop(img, ((0, 25), (210, 0), (0,0)), copy=False)
  imsave('/content/dataset_CD_UC/ulcerative-colitis/'+url.split('/')[4], crop_img)

In [ ]:
#Crop frames
for url in glob.glob('/content/frames-crop/*.jpg'):
  img = imread(url)
  crop_img = crop(img, ((0, 25), (210, 0), (0,0)), copy=False)
  imsave('/content/frames/'+url.split('/')[3], crop_img)

In [ ]:
!rm -r '/content/dataset_CD_UC/ulcerative-colitis-crop/'
!rm -r '/content/frames-crop/'

<h1>Split dataset</h1>

In [ ]:
data_dir = pathlib.Path('/content/dataset_CD_UC')
data_dir_frames = pathlib.Path('/content/frames')

# Input size initial
batch_size = 32
#IMG_SIZE = (227, 227)
#IMG_SIZE = (299, 299)
IMG_SIZE = (224, 224)

train_ds = image_dataset_from_directory(
  data_dir,
  validation_split=0.7,
  subset="training",
  seed=123,
  shuffle=True,
  image_size=IMG_SIZE,
  batch_size=batch_size)

val_ds = image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="validation",
  seed=123,
  shuffle=True,
  image_size=IMG_SIZE,
  batch_size=batch_size)

val_batches = cardinality(val_ds)
test_ds = val_ds.take((2*val_batches) // 3)    #Take 20% for test
val_ds = val_ds.skip((2*val_batches) // 3)   #Take 10% for validation

class_names = train_ds.class_names

print(class_names)

In [ ]:
frames_dataset = image_dataset_from_directory(data_dir_frames,image_size=IMG_SIZE,batch_size=batch_size)
test_ds = test_ds.concatenate(frames_dataset.skip(34))    #Take the last 30 frames of the ulcerative colitis disease for concatenating to the test dataset
train_ds = train_ds.concatenate(frames_dataset.take(34))  #Take the first 34 frames of the ulcerative colitis disease for concatenate to the train dataset

In [ ]:
def class_distribution(dataset):
    class_values = []
    total_batches = dataset.__len__().numpy()
    for batch, element in enumerate(dataset.as_numpy_iterator()):
        if batch+1 == total_batches:
            for i in range(len(element[1])):
                class_values.append(class_names[int(element[1][i])])
        else:
            for i in range(batch_size):
                class_values.append(class_names[int(element[1][i])])

    class_n, frequency = np.unique(np.array(class_values), return_counts=True)
    count = pd.DataFrame(columns=["Classes","Count"])
    count["Classes"] = class_n
    count["Count"] = frequency
    return count, class_values

dataset1 = image_dataset_from_directory(data_dir)

df_class_dist, class_df_values_val = class_distribution(dataset1)
plt.figure(figsize=(25, 12))
sns.barplot(data=df_class_dist, y="Count" ,x="Classes");
plt.show()

In [ ]:
plt.figure(figsize=(25, 15))
for images, labels in train_ds.take(1):
    for i in range(10):
        ax = plt.subplot(2, 10, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

<h1>Classificiation</h1>

In [ ]:
data_augmentation = Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomContrast(0.15),
  layers.RandomZoom(-.2,-.3)
])

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)

In [ ]:
images, labels = tuple(zip(*test_ds))

true_images = np.concatenate(images)
true_labels = np.concatenate(labels)

# AlexNet

In [ ]:
model = Sequential([
    data_augmentation,
    layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])

In [ ]:
model.build((None, 227, 227,3))
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=1e-5), metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs=100, validation_data=test_ds, callbacks=[early_stopping])

In [ ]:
model.evaluate(val_ds)

In [ ]:
model_history = pd.DataFrame(history.history)
model_history.loc[:, ['loss', 'val_loss']].plot()
model_history.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
print('Accuracy for Train: {:.4f}'.format(model_history['accuracy'].mean()))
print('Loss for Train: {:.4f}'.format(model_history['loss'].mean()))
print('Accuracy for Test: {:.4f}'.format(model_history['val_accuracy'].mean()))
print('Loss for Test: {:.4f}'.format(model_history['val_loss'].mean()))

In [ ]:
predict_labels = np.argmax(model.predict(true_images), axis=1)

In [ ]:
conf_matrix = pd.DataFrame(confusion_matrix(true_labels, predict_labels), columns=['crohn-disease', 'ulcerative-colitis'], index=['crohn-disease', 'ulcerative-colitis'])
ax = sns.heatmap(conf_matrix, annot=True, fmt='', cmap='Blues')
ax.set(xlabel='True Class', ylabel='Predicted Class')
plt.show()

In [ ]:
print("Precision: {:0.4f}".format(precision_score(true_labels, predict_labels)))
print("Recall: {:0.4f}".format(recall_score(true_labels, predict_labels)))
print("F1-Score: {:0.4f}".format(f1_score(true_labels, predict_labels)))
print("AUC: {:0.4f}".format(roc_auc_score(true_labels, predict_labels)))
print("MCC: {:0.4f}".format(matthews_corrcoef(true_labels, predict_labels)))

<h1>Interpretability</h1>

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
def read_and_transform_img(url):

    img = imread(url)
    img = resize(img, (227,227))
    
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)

    return img

image1 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00021.jpg')
image2 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/00064260-95ca-47fc-9103-2b526f59fada.jpg')
image3 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/007bcee7-a272-4e74-b10d-78d4d7816678.jpg')
image4 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/00909788-4bd5-42d7-bd4e-729897520cc5.jpg')
image5 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00033.jpg')
image6 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00169.jpg')
image7 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00178.jpg')
image8 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/0954d7e1-cbbc-4b1e-b0db-e5db9a3579fd.jpg')
image9 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/108e5950-a265-4df3-9f58-5867fe75a8c5.jpg')

In [ ]:
explanation2 = explainer.explain_instance(image2[0].astype('double'), model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model.predict(image2)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation2.get_image_and_mask(explanation2.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation2.get_image_and_mask(explanation2.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image2[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation3 = explainer.explain_instance(image3[0].astype('double'), model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model.predict(image3)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation3.get_image_and_mask(explanation3.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation3.get_image_and_mask(explanation3.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image3[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation4 = explainer.explain_instance(image4[0].astype('double'), model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model.predict(image4)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation4.get_image_and_mask(explanation4.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation4.get_image_and_mask(explanation4.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image4[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation1 = explainer.explain_instance(image1[0].astype('double'), model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model.predict(image1)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation1.get_image_and_mask(explanation1.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation1.get_image_and_mask(explanation1.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image1[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation5 = explainer.explain_instance(image5[0].astype('double'), model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model.predict(image5)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation5.get_image_and_mask(explanation5.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation5.get_image_and_mask(explanation5.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image5[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation6 = explainer.explain_instance(image6[0].astype('double'), model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model.predict(image6)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation6.get_image_and_mask(explanation6.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation6.get_image_and_mask(explanation6.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image6[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation7 = explainer.explain_instance(image7[0].astype('double'), model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model.predict(image7)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation7.get_image_and_mask(explanation7.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation7.get_image_and_mask(explanation7.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image7[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation8 = explainer.explain_instance(image8[0].astype('double'), model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model.predict(image8)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation8.get_image_and_mask(explanation8.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation8.get_image_and_mask(explanation8.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image8[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation9 = explainer.explain_instance(image9[0].astype('double'), model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model.predict(image9)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation9.get_image_and_mask(explanation9.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation9.get_image_and_mask(explanation9.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image9[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

# ResNet50

In [ ]:
inputs = Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input_resnet(x)
outputs = ResNet50(weights=None, classes=len(class_names))(x)
model_resnet50 = Model(inputs, outputs)

In [ ]:
model_resnet50.summary()

In [ ]:
model_resnet50.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=1e-5), metrics=['accuracy'])

In [ ]:
history_resnet50 = model_resnet50.fit(train_ds, epochs=100, validation_data=test_ds, callbacks=[early_stopping])

In [ ]:
model_resnet50.evaluate(val_ds)

In [ ]:
model_history_resnet50 = pd.DataFrame(history_resnet50.history)
model_history_resnet50.loc[:, ['loss', 'val_loss']].plot()
model_history_resnet50.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
print('Accuracy for Train: {:.4f}'.format(model_history_resnet50['accuracy'].mean()))
print('Loss for Train: {:.4f}'.format(model_history_resnet50['loss'].mean()))
print('Accuracy for Test: {:.4f}'.format(model_history_resnet50['val_accuracy'].mean()))
print('Loss for Test: {:.4f}'.format(model_history_resnet50['val_loss'].mean()))

In [ ]:
predict_labels_resnet50 = np.argmax(model_resnet50.predict(true_images), axis=1)

In [ ]:
conf_matrix_resnet50 = pd.DataFrame(confusion_matrix(true_labels, predict_labels_resnet50), columns=['crohn-disease', 'ulcerative-colitis'], index=['crohn-disease', 'ulcerative-colitis'])
ax = sns.heatmap(conf_matrix_resnet50, annot=True, fmt='', cmap='Blues')
ax.set(xlabel='True Class', ylabel='Predicted Class')
plt.show()

In [ ]:
print("Precision: {:0.4f}".format(precision_score(true_labels, predict_labels_resnet50)))
print("Recall: {:0.4f}".format(recall_score(true_labels, predict_labels_resnet50)))
print("F1-Score: {:0.4f}".format(f1_score(true_labels, predict_labels_resnet50)))
print("AUC: {:0.4f}".format(roc_auc_score(true_labels, predict_labels_resnet50)))
print("MCC: {:0.4f}".format(matthews_corrcoef(true_labels, predict_labels_resnet50)))

<h1>Interpretability</h1>

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
def read_and_transform_img(url):

    img = imread(url)
    img = resize(img, (224,224))
    
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)

    return img

image1 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00021.jpg')
image2 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/00064260-95ca-47fc-9103-2b526f59fada.jpg')
image3 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/007bcee7-a272-4e74-b10d-78d4d7816678.jpg')
image4 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/00909788-4bd5-42d7-bd4e-729897520cc5.jpg')
image5 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00033.jpg')
image6 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00169.jpg')
image7 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00178.jpg')
image8 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/0954d7e1-cbbc-4b1e-b0db-e5db9a3579fd.jpg')
image9 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/108e5950-a265-4df3-9f58-5867fe75a8c5.jpg')

In [ ]:
explanation2 = explainer.explain_instance(image2[0].astype('double'), model_resnet50.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_resnet50.predict(image2)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation2.get_image_and_mask(explanation2.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation2.get_image_and_mask(explanation2.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image2[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation3 = explainer.explain_instance(image3[0].astype('double'), model_resnet50.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_resnet50.predict(image3)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation3.get_image_and_mask(explanation3.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation3.get_image_and_mask(explanation3.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image3[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation4 = explainer.explain_instance(image4[0].astype('double'), model_resnet50.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_resnet50.predict(image4)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation4.get_image_and_mask(explanation4.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation4.get_image_and_mask(explanation4.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image4[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation1 = explainer.explain_instance(image1[0].astype('double'), model_resnet50.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_resnet50.predict(image1)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation1.get_image_and_mask(explanation1.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation1.get_image_and_mask(explanation1.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image1[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation5 = explainer.explain_instance(image5[0].astype('double'), model_resnet50.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_resnet50.predict(image5)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation5.get_image_and_mask(explanation5.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation5.get_image_and_mask(explanation5.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image5[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation6 = explainer.explain_instance(image6[0].astype('double'), model_resnet50.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_resnet50.predict(image6)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation6.get_image_and_mask(explanation6.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation6.get_image_and_mask(explanation6.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image6[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation7 = explainer.explain_instance(image7[0].astype('double'), model_resnet50.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_resnet50.predict(image7)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation7.get_image_and_mask(explanation7.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation7.get_image_and_mask(explanation7.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image7[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation8 = explainer.explain_instance(image8[0].astype('double'), model_resnet50.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_resnet50.predict(image8)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation8.get_image_and_mask(explanation8.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation8.get_image_and_mask(explanation8.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image8[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation9 = explainer.explain_instance(image9[0].astype('double'), model_resnet50.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_resnet50.predict(image9)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation9.get_image_and_mask(explanation9.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation9.get_image_and_mask(explanation9.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image9[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

# VGG16

In [ ]:
inputs = Input(shape=(224,224,3))
x = data_augmentation(inputs)
x = preprocess_input_vgg16(x)
outputs = VGG16(weights=None, classes=2)(x)
model_vgg16 = Model(inputs, outputs)

In [ ]:
model_vgg16.summary()

In [ ]:
model_vgg16.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=1e-5), metrics=['accuracy'])

In [ ]:
history_vgg = model_vgg16.fit(train_ds, epochs=100, validation_data=test_ds, callbacks=[early_stopping])

In [ ]:
model_vgg16.evaluate(val_ds)

In [ ]:
model_history_vgg = pd.DataFrame(history_vgg.history)
model_history_vgg.loc[:, ['loss', 'val_loss']].plot()
model_history_vgg.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
print('Accuracy for Train: {:.4f}'.format(model_history_vgg['accuracy'].mean()))
print('Loss for Train: {:.4f}'.format(model_history_vgg['loss'].mean()))
print('Accuracy for Test: {:.4f}'.format(model_history_vgg['val_accuracy'].mean()))
print('Loss for Test: {:.4f}'.format(model_history_vgg['val_loss'].mean()))

In [ ]:
predict_labels_vgg = np.argmax(model_vgg16.predict(true_images), axis=1)

In [ ]:
conf_matrix_vgg = pd.DataFrame(confusion_matrix(true_labels, predict_labels_vgg), columns=['crohn-disease', 'ulcerative-colitis'], index=['crohn-disease', 'ulcerative-colitis'])
ax = sns.heatmap(conf_matrix_vgg, annot=True, fmt='', cmap='Blues')
ax.set(xlabel='True Class', ylabel='Predicted Class')
plt.show()

In [ ]:
print("Precision: {:0.4f}".format(precision_score(true_labels, predict_labels_vgg)))
print("Recall: {:0.4f}".format(recall_score(true_labels, predict_labels_vgg)))
print("F1-Score: {:0.4f}".format(f1_score(true_labels, predict_labels_vgg)))
print("AUC: {:0.4f}".format(roc_auc_score(true_labels, predict_labels_vgg)))
print("MCC: {:0.4f}".format(matthews_corrcoef(true_labels, predict_labels_vgg)))

<h1>Interpretability</h1>

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
def read_and_transform_img(url):

    img = imread(url)
    img = resize(img, (224,224))
    
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)

    return img

image1 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00021.jpg')
image2 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/00064260-95ca-47fc-9103-2b526f59fada.jpg')
image3 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/007bcee7-a272-4e74-b10d-78d4d7816678.jpg')
image4 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/00909788-4bd5-42d7-bd4e-729897520cc5.jpg')
image5 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00033.jpg')
image6 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00169.jpg')
image7 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00178.jpg')
image8 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/0954d7e1-cbbc-4b1e-b0db-e5db9a3579fd.jpg')
image9 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/108e5950-a265-4df3-9f58-5867fe75a8c5.jpg')

In [ ]:
explanation2 = explainer.explain_instance(image2[0].astype('double'), model_vgg16.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_vgg16.predict(image2)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation2.get_image_and_mask(explanation2.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation2.get_image_and_mask(explanation2.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image2[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation3 = explainer.explain_instance(image3[0].astype('double'), model_vgg16.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_vgg16.predict(image3)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation3.get_image_and_mask(explanation3.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation3.get_image_and_mask(explanation3.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image3[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation4 = explainer.explain_instance(image4[0].astype('double'), model_vgg16.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_vgg16.predict(image4)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation4.get_image_and_mask(explanation4.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation4.get_image_and_mask(explanation4.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image4[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation1 = explainer.explain_instance(image1[0].astype('double'), model_vgg16.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_vgg16.predict(image1)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation1.get_image_and_mask(explanation1.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation1.get_image_and_mask(explanation1.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image1[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation5 = explainer.explain_instance(image5[0].astype('double'), model_vgg16.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_vgg16.predict(image5)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation5.get_image_and_mask(explanation5.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation5.get_image_and_mask(explanation5.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image5[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation6 = explainer.explain_instance(image6[0].astype('double'), model_vgg16.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_vgg16.predict(image6)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation6.get_image_and_mask(explanation6.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation6.get_image_and_mask(explanation6.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image6[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation7 = explainer.explain_instance(image7[0].astype('double'), model_vgg16.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_vgg16.predict(image7)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation7.get_image_and_mask(explanation7.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation7.get_image_and_mask(explanation7.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image7[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation8 = explainer.explain_instance(image8[0].astype('double'), model_vgg16.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_vgg16.predict(image8)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation8.get_image_and_mask(explanation8.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation8.get_image_and_mask(explanation8.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image8[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation9 = explainer.explain_instance(image9[0].astype('double'), model_vgg16.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_vgg16.predict(image9)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation9.get_image_and_mask(explanation9.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation9.get_image_and_mask(explanation9.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image9[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

# InceptionV3

In [ ]:
inputs = Input(shape=(299,299,3))
x = data_augmentation(inputs)
x = preprocess_input_inceptionV3(x)
outputs = InceptionV3(weights=None, classes=len(class_names))(x)
model_inceptionV3 = Model(inputs, outputs)

In [ ]:
model_inceptionV3.summary()

In [ ]:
model_inceptionV3.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=1e-5), metrics=['accuracy'])

In [ ]:
history_inceptionV3 = model_inceptionV3.fit(train_ds, epochs=100, validation_data=test_ds, callbacks=[early_stopping])

In [ ]:
model_inceptionV3.evaluate(val_ds)

In [ ]:
model_history_inceptionV3 = pd.DataFrame(history_inceptionV3.history)
model_history_inceptionV3.loc[:, ['loss', 'val_loss']].plot()
model_history_inceptionV3.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
print('Accuracy for Train: {:.4f}'.format(model_history_inceptionV3['accuracy'].mean()))
print('Loss for Train: {:.4f}'.format(model_history_inceptionV3['loss'].mean()))
print('Accuracy for Test: {:.4f}'.format(model_history_inceptionV3['val_accuracy'].mean()))
print('Loss for Test: {:.4f}'.format(model_history_inceptionV3['val_loss'].mean()))

In [ ]:
predict_labels_inceptionV3 = np.argmax(model_inceptionV3.predict(true_images), axis=1)

In [ ]:
conf_matrix_inceptionV3 = pd.DataFrame(confusion_matrix(true_labels, predict_labels_inceptionV3), columns=['crohn-disease', 'ulcerative-colitis'], index=['crohn-disease', 'ulcerative-colitis'])
ax = sns.heatmap(conf_matrix_inceptionV3, annot=True, fmt='', cmap='Blues')
ax.set(xlabel='True Class', ylabel='Predicted Class')
plt.show()

In [ ]:
print("Precision: {:0.4f}".format(precision_score(true_labels, predict_labels_inceptionV3)))
print("Recall: {:0.4f}".format(recall_score(true_labels, predict_labels_inceptionV3)))
print("F1-Score: {:0.4f}".format(f1_score(true_labels, predict_labels_inceptionV3)))
print("AUC: {:0.4f}".format(roc_auc_score(true_labels, predict_labels_inceptionV3)))
print("MCC: {:0.4f}".format(matthews_corrcoef(true_labels, predict_labels_inceptionV3)))

<h1>Interpretability</h1>

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
def read_and_transform_img(url):

    img = imread(url)
    img = resize(img, (299,299))
    
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)

    return img

image1 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00021.jpg')
image2 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/00064260-95ca-47fc-9103-2b526f59fada.jpg')
image3 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/007bcee7-a272-4e74-b10d-78d4d7816678.jpg')
image4 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/00909788-4bd5-42d7-bd4e-729897520cc5.jpg')
image5 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00033.jpg')
image6 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00169.jpg')
image7 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00178.jpg')
image8 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/0954d7e1-cbbc-4b1e-b0db-e5db9a3579fd.jpg')
image9 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/108e5950-a265-4df3-9f58-5867fe75a8c5.jpg')

In [ ]:
explanation2 = explainer.explain_instance(image2[0].astype('double'), model_inceptionV3.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_inceptionV3.predict(image2)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation2.get_image_and_mask(explanation2.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation2.get_image_and_mask(explanation2.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image2[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation3 = explainer.explain_instance(image3[0].astype('double'), model_inceptionV3.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_inceptionV3.predict(image3)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation3.get_image_and_mask(explanation3.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation3.get_image_and_mask(explanation3.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image3[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation4 = explainer.explain_instance(image4[0].astype('double'), model_inceptionV3.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_inceptionV3.predict(image4)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation4.get_image_and_mask(explanation4.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation4.get_image_and_mask(explanation4.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image4[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation1 = explainer.explain_instance(image1[0].astype('double'), model_inceptionV3.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_inceptionV3.predict(image1)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation1.get_image_and_mask(explanation1.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation1.get_image_and_mask(explanation1.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image1[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation5 = explainer.explain_instance(image5[0].astype('double'), model_inceptionV3.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_inceptionV3.predict(image5)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation5.get_image_and_mask(explanation5.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation5.get_image_and_mask(explanation5.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image5[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation6 = explainer.explain_instance(image6[0].astype('double'), model_inceptionV3.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_inceptionV3.predict(image6)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation6.get_image_and_mask(explanation6.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation6.get_image_and_mask(explanation6.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image6[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation7 = explainer.explain_instance(image7[0].astype('double'), model_inceptionV3.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_inceptionV3.predict(image7)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation7.get_image_and_mask(explanation7.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation7.get_image_and_mask(explanation7.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image7[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation8 = explainer.explain_instance(image8[0].astype('double'), model_inceptionV3.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_inceptionV3.predict(image8)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation8.get_image_and_mask(explanation8.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation8.get_image_and_mask(explanation8.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image8[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation9 = explainer.explain_instance(image9[0].astype('double'), model_inceptionV3.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_inceptionV3.predict(image9)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation9.get_image_and_mask(explanation9.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation9.get_image_and_mask(explanation9.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image9[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

# Resnet50+MobileNetV2

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)

base_model = ResNet50(input_shape=IMG_SHAPE, include_top=False)  #include_top=False doesn't include classification layers for feature extraction

In [ ]:
base_model.trainable = False    #This prevents the weights in a given layer from being updated during training

In [ ]:
model_classification = MobileNetV2()
model_classification.summary()

In [ ]:
inputs = Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input_resnet(x)
x = base_model(x, training=False)
x = layers.Conv2D(1280,1)(x)
x = model_classification.get_layer(name='global_average_pooling2d')(x)
x = model_classification.get_layer(name='predictions')(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)
model_2 = Model(inputs, outputs)

In [ ]:
model_2.summary()

In [ ]:
model_2.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=1e-5), metrics=['accuracy'])

In [ ]:
history_2 = model_2.fit(train_ds, epochs=100, validation_data=test_ds, callbacks=[early_stopping])

In [ ]:
model_2.evaluate(val_ds)

In [ ]:
model_history_2 = pd.DataFrame(history_2.history)
model_history_2.loc[:, ['loss', 'val_loss']].plot()
model_history_2.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
print('Accuracy for Train: {:.4f}'.format(model_history_2['accuracy'].mean()))
print('Loss for Train: {:.4f}'.format(model_history_2['loss'].mean()))
print('Accuracy for Test: {:.4f}'.format(model_history_2['val_accuracy'].mean()))
print('Loss for Test: {:.4f}'.format(model_history_2['val_loss'].mean()))

In [ ]:
predict_labels_2 = np.argmax(model_2.predict(true_images), axis=1)

In [ ]:
conf_matrix_2 = pd.DataFrame(confusion_matrix(true_labels, predict_labels_2), columns=['crohn-disease', 'ulcerative-colitis'], index=['crohn-disease', 'ulcerative-colitis'])
ax = sns.heatmap(conf_matrix_2, annot=True, fmt='', cmap='Blues')
ax.set(xlabel='True Class', ylabel='Predicted Class')
plt.show()

In [ ]:
print("Precision: {:0.4f}".format(precision_score(true_labels, predict_labels_2)))
print("Recall: {:0.4f}".format(recall_score(true_labels, predict_labels_2)))
print("F1-Score: {:0.4f}".format(f1_score(true_labels, predict_labels_2)))
print("AUC: {:0.4f}".format(roc_auc_score(true_labels, predict_labels_2)))
print("MCC: {:0.4f}".format(matthews_corrcoef(true_labels, predict_labels_2)))

<h1>Interpretability</h1>

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
def read_and_transform_img(url):

    img = imread(url)
    img = resize(img, (224,224))
    
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)

    return img

image1 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00021.jpg')
image2 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/00064260-95ca-47fc-9103-2b526f59fada.jpg')
image3 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/007bcee7-a272-4e74-b10d-78d4d7816678.jpg')
image4 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/00909788-4bd5-42d7-bd4e-729897520cc5.jpg')
image5 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00033.jpg')
image6 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00169.jpg')
image7 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00178.jpg')
image8 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/0954d7e1-cbbc-4b1e-b0db-e5db9a3579fd.jpg')
image9 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/108e5950-a265-4df3-9f58-5867fe75a8c5.jpg')

In [ ]:
explanation2 = explainer.explain_instance(image2[0].astype('double'), model_2.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_2.predict(image2)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation2.get_image_and_mask(explanation2.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation2.get_image_and_mask(explanation2.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image2[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation3 = explainer.explain_instance(image3[0].astype('double'), model_2.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_2.predict(image3)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation3.get_image_and_mask(explanation3.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation3.get_image_and_mask(explanation3.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image3[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation4 = explainer.explain_instance(image4[0].astype('double'), model_2.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_2.predict(image4)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation4.get_image_and_mask(explanation4.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation4.get_image_and_mask(explanation4.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image4[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation1 = explainer.explain_instance(image1[0].astype('double'), model_2.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_2.predict(image1)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation1.get_image_and_mask(explanation1.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation1.get_image_and_mask(explanation1.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image1[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation5 = explainer.explain_instance(image5[0].astype('double'), model_2.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_2.predict(image5)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation5.get_image_and_mask(explanation5.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation5.get_image_and_mask(explanation5.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image5[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation6 = explainer.explain_instance(image6[0].astype('double'), model_2.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_2.predict(image6)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation6.get_image_and_mask(explanation6.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation6.get_image_and_mask(explanation6.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image6[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation7 = explainer.explain_instance(image7[0].astype('double'), model_2.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_2.predict(image7)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation7.get_image_and_mask(explanation7.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation7.get_image_and_mask(explanation7.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image7[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation8 = explainer.explain_instance(image8[0].astype('double'), model_2.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_2.predict(image8)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation8.get_image_and_mask(explanation8.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation8.get_image_and_mask(explanation8.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image8[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation9 = explainer.explain_instance(image9[0].astype('double'), model_2.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(model_2.predict(image9)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation9.get_image_and_mask(explanation9.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation9.get_image_and_mask(explanation9.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image9[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

# CNN+LSTM

In [ ]:
inputs = Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
x = layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
x = layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
x = layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
x = layers.TimeDistributed(layers.Flatten())(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.LSTM(100, return_sequences=True)(x)
x = layers.LSTM(100, return_sequences=True)(x)
x = layers.LSTM(50)(x)
x = layers.Dense(50, activation='relu')(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(50, activation='relu')(x)
x = layers.Dense(50, activation='relu')(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)
hybrid_model = Model(inputs, outputs)

In [ ]:
hybrid_model.summary()

In [ ]:
hybrid_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=1e-5), metrics=['accuracy'])

In [ ]:
history_hybrid_model = hybrid_model.fit(train_ds, epochs=100, validation_data=test_ds, callbacks=[early_stopping])

In [ ]:
hybrid_model.evaluate(val_ds)

In [ ]:
model_history_hybrid_model = pd.DataFrame(history_hybrid_model.history)
model_history_hybrid_model.loc[:, ['loss', 'val_loss']].plot()
model_history_hybrid_model.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
print('Accuracy for Train: {:.4f}'.format(model_history_hybrid_model['accuracy'].mean()))
print('Loss for Train: {:.4f}'.format(model_history_hybrid_model['loss'].mean()))
print('Accuracy for Test: {:.4f}'.format(model_history_hybrid_model['val_accuracy'].mean()))
print('Loss for Test: {:.4f}'.format(model_history_hybrid_model['val_loss'].mean()))

In [ ]:
predict_labels_hybrid_model = np.argmax(hybrid_model.predict(true_images), axis=1)

In [ ]:
conf_matrix = pd.DataFrame(confusion_matrix(true_labels, predict_labels_hybrid_model), columns=['crohn-disease', 'ulcerative-colitis'], index=['crohn-disease', 'ulcerative-colitis'])
ax = sns.heatmap(conf_matrix, annot=True, fmt='', cmap='Blues')
ax.set(xlabel='True Class', ylabel='Predicted Class')
plt.show()

In [ ]:
print("Precision: {:0.4f}".format(precision_score(true_labels, predict_labels_hybrid_model)))
print("Recall: {:0.4f}".format(recall_score(true_labels, predict_labels_hybrid_model)))
print("F1-Score: {:0.4f}".format(f1_score(true_labels, predict_labels_hybrid_model)))
print("AUC: {:0.4f}".format(roc_auc_score(true_labels, predict_labels_hybrid_model)))
print("MCC: {:0.4f}".format(matthews_corrcoef(true_labels, predict_labels_hybrid_model)))

<h1>Interpretability</h1>

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
def read_and_transform_img(url):

    img = imread(url)
    img = resize(img, (224,224))
    
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)

    return img

image1 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00021.jpg')
image2 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/00064260-95ca-47fc-9103-2b526f59fada.jpg')
image3 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/007bcee7-a272-4e74-b10d-78d4d7816678.jpg')
image4 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/00909788-4bd5-42d7-bd4e-729897520cc5.jpg')
image5 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00033.jpg')
image6 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00169.jpg')
image7 = read_and_transform_img('/content/dataset_CD_UC/crohn-disease/00178.jpg')
image8 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/0954d7e1-cbbc-4b1e-b0db-e5db9a3579fd.jpg')
image9 = read_and_transform_img('/content/dataset_CD_UC/ulcerative-colitis/108e5950-a265-4df3-9f58-5867fe75a8c5.jpg')

In [ ]:
explanation2 = explainer.explain_instance(image2[0].astype('double'), hybrid_model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(hybrid_model.predict(image2)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation2.get_image_and_mask(explanation2.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation2.get_image_and_mask(explanation2.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image2[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation3 = explainer.explain_instance(image3[0].astype('double'), hybrid_model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(hybrid_model.predict(image3)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation3.get_image_and_mask(explanation3.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation3.get_image_and_mask(explanation3.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image3[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation4 = explainer.explain_instance(image4[0].astype('double'), hybrid_model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(hybrid_model.predict(image4)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation4.get_image_and_mask(explanation4.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation4.get_image_and_mask(explanation4.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image4[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation1 = explainer.explain_instance(image1[0].astype('double'), hybrid_model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(hybrid_model.predict(image1)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation1.get_image_and_mask(explanation1.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation1.get_image_and_mask(explanation1.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image1[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation5 = explainer.explain_instance(image5[0].astype('double'), hybrid_model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(hybrid_model.predict(image5)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation5.get_image_and_mask(explanation5.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation5.get_image_and_mask(explanation5.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image5[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation6 = explainer.explain_instance(image6[0].astype('double'), hybrid_model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(hybrid_model.predict(image6)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation6.get_image_and_mask(explanation6.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation6.get_image_and_mask(explanation6.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image6[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation7 = explainer.explain_instance(image7[0].astype('double'), hybrid_model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(hybrid_model.predict(image7)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Crohn disease
temp_1, mask_1 = explanation7.get_image_and_mask(explanation7.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation7.get_image_and_mask(explanation7.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image7[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation8 = explainer.explain_instance(image8[0].astype('double'), hybrid_model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(hybrid_model.predict(image8)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation8.get_image_and_mask(explanation8.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation8.get_image_and_mask(explanation8.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image8[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

In [ ]:
explanation9 = explainer.explain_instance(image9[0].astype('double'), hybrid_model.predict, top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
pred = 'ulcerative colitis' if np.argmax(hybrid_model.predict(image9)) == 1 else 'cronh disease'
pred

In [ ]:
#Interpretabillity of the images relative to Ulcerative Colitis
temp_1, mask_1 = explanation9.get_image_and_mask(explanation9.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
temp_2, mask_2 = explanation9.get_image_and_mask(explanation9.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2))
ax3.imshow(image9[0])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')